In [2]:
import pandas as pd
import math

In [3]:
sheet_id = "1KaB4ti8NissgX45bOuRpnoE07qknv7VQk_ECJ1zpJt0"
sheet_name = "Star\ Wars\ Universe"
url = f"https://docs.google.com/spreadsheets/d/{sheet_id}/gviz/tq?tqx=out:csv"
df = pd.read_csv('StarWarsUniverse.csv', keep_default_na=False)

In [5]:
def parse_date(date_str):
    if date_str == '':
        return
    parts = date_str.split('-')
    parts = [p.strip() for p in parts]
    if len(parts) == 1:
        [year, label] = parts[0].split(' ')
        year = float(year)
        if label == 'BBY':
            year *= -1 
        return [year]
    elif len(parts) == 2:
        [p1, p2] = parts
        p1_parts = p1.split(' ')
        y1 = float(p1_parts[0])
        if len(p1_parts) == 2 and p1_parts[1] == 'BBY':
            y1 *= -1
        p2_parts = p2.split(' ')
        y2 = float(p2_parts[0])
        if p2_parts[1] == 'BBY':
            y2 *= -1
            if y1 > 0:
                y1 *= -1
        return [y1, y2]
    return None

def parse_rdate(date_str):
    if date_str == '':
        return
    parts = date_str.split('-')
    parts = [p.strip() for p in parts]
    if len(parts) == 1:
        return [ int(parts[0].split(' ')[0].strip()) ]
    else:
        return [ int(parts[0]), int(parts[1]) ]

names = list(df['Name'])
dates = list(df['Date (Galactic Calendar)'])
dates = [parse_date(str(d)) for d in dates]
rdates = list(df['Release date'])
rdates = [parse_rdate(str(d)) for d in rdates]
types = list(df['Type'])

n = len(names)
data = []
for i in range(n):
    name = names[i]
    fname = "".join(x for x in name if x.isalnum())
    item = { 
        'name': name,
        'releaseDate': rdates[i],
        'type': types[i],
        'thumbnail': f"/images/{fname}.jpg",
        'params': {}
    }
    if dates[i] is None:
        continue
    if len(dates[i]) == 1:
        item['year'] = dates[i][0]
    else:
        item['startYear'] = dates[i][0]
        item['endYear'] = dates[i][1]
    data.append(item)

import json
save_path = '../src/js/data2.json'
with open(save_path, 'w') as f:
    json.dump(data, f)